In [181]:
import pandas as pd
import numpy as np

In [165]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [166]:
df_train.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


### Feature engineering

In [167]:
def create_sinusoidal_transformation_year_month_day(df, col_name, year, month, day, period):
    """
    Adds sinusoidal transformation columns (sin and cos) for year, month, day.
    """
    df[f'{col_name}_sin'] = np.sin(2 * np.pi * df[year] * df[month] * df[day] / period)
    df[f'{col_name}_cos'] = np.cos(2 * np.pi * df[year] * df[month] * df[day] / period)
    return df


In [168]:
def add_time_features(data, date_col='date'):
    df = data.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col).reset_index(drop=True)

    # Time-based features
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day

    df = create_sinusoidal_transformation_year_month_day(df, 'date', "year", "month", "day", 12)

    return df

In [ ]:
# lag features
def add_lag_features(data):




    return

In [169]:
df_train = add_time_features(df_train)

In [170]:
df_train

,id,date,country,store,product,num_sold,year,month,day,date_sin,date_cos
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN,2010,1,1,8.721095e-14,-1.0
1,65,2010-01-01,Norway,Stickers for Less,Holographic Goose,579.0,2010,1,1,8.721095e-14,-1.0
2,64,2010-01-01,Norway,Discount Stickers,Kerneler Dark Mode,911.0,2010,1,1,8.721095e-14,-1.0
3,63,2010-01-01,Norway,Discount Stickers,Kerneler,791.0,2010,1,1,8.721095e-14,-1.0
4,62,2010-01-01,Norway,Discount Stickers,Kaggle Tiers,1525.0,2010,1,1,8.721095e-14,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
230125,230065,2016-12-31,Finland,Premium Sticker Mart,Holographic Goose,349.0,2016,12,31,1.311460e-11,1.0
230126,230064,2016-12-31,Finland,Stickers for Less,Kerneler Dark Mode,1012.0,2016,12,31,1.311460e-11,1.0
230127,230063,2016-12-31,Finland,Stickers for Less,Kerneler,846.0,2016,12,31,1.311460e-11,1.0
230128,230060,2016-12-31,Finland,Stickers for Less,Holographic Goose,289.0,2016,12,31,1.311460e-11,1.0


### Imputation

Monthly average sales per product per store per country

In [171]:
def imputation(data, group_by):
    df = data.copy()

    df['num_sold'] = df['num_sold'].fillna(0)

    df_temp = df.groupby(group_by)['num_sold'].mean().reset_index(name='avg_sold').round(0)

    df_merge = pd.merge(df_train, df_temp, how='left', on=group_by)

    df_merge['num_sold'] = np.where(df_merge['num_sold'].isna(), df_merge['avg_sold'], df_merge['num_sold'])

    return df_merge

In [172]:
group_by = ['country','store','product', 'month']
df_train = imputation(df_train, group_by)

In [173]:
df_train.head()

,id,date,country,store,product,num_sold,year,month,day,date_sin,date_cos,avg_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,0.0,2010,1,1,8.721095e-14,-1.0,0.0
1,65,2010-01-01,Norway,Stickers for Less,Holographic Goose,579.0,2010,1,1,8.721095e-14,-1.0,429.0
2,64,2010-01-01,Norway,Discount Stickers,Kerneler Dark Mode,911.0,2010,1,1,8.721095e-14,-1.0,748.0
3,63,2010-01-01,Norway,Discount Stickers,Kerneler,791.0,2010,1,1,8.721095e-14,-1.0,581.0
4,62,2010-01-01,Norway,Discount Stickers,Kaggle Tiers,1525.0,2010,1,1,8.721095e-14,-1.0,1129.0


In [174]:
df_train.isna().sum()

id          0
date        0
country     0
store       0
product     0
num_sold    0
year        0
month       0
day         0
date_sin    0
date_cos    0
avg_sold    0
dtype: int64

### Encoding

one-hot encoding

In [180]:
categorical_col = df_train.select_dtypes('object').columns.to_list()
categorical_col

['country', 'store', 'product']

In [187]:
df_encoded = pd.get_dummies(df_train, columns=categorical_col, drop_first=False, dtype=int)

In [188]:
df_encoded

,id,date,num_sold,year,month,day,date_sin,date_cos,avg_sold,country_Canada,...,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
0,0,2010-01-01,0.0,2010,1,1,8.721095e-14,-1.0,0.0,1,...,0,0,1,0,0,1,0,0,0,0
1,65,2010-01-01,579.0,2010,1,1,8.721095e-14,-1.0,429.0,0,...,1,0,0,0,1,1,0,0,0,0
2,64,2010-01-01,911.0,2010,1,1,8.721095e-14,-1.0,748.0,0,...,1,0,1,0,0,0,0,0,0,1
3,63,2010-01-01,791.0,2010,1,1,8.721095e-14,-1.0,581.0,0,...,1,0,1,0,0,0,0,0,1,0
4,62,2010-01-01,1525.0,2010,1,1,8.721095e-14,-1.0,1129.0,0,...,1,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230065,2016-12-31,349.0,2016,12,31,1.311460e-11,1.0,282.0,0,...,0,0,0,1,0,1,0,0,0,0
230126,230064,2016-12-31,1012.0,2016,12,31,1.311460e-11,1.0,757.0,0,...,0,0,0,0,1,0,0,0,0,1
230127,230063,2016-12-31,846.0,2016,12,31,1.311460e-11,1.0,709.0,0,...,0,0,0,0,1,0,0,0,1,0
230128,230060,2016-12-31,289.0,2016,12,31,1.311460e-11,1.0,237.0,0,...,0,0,0,0,1,1,0,0,0,0
